In [ ]:
import rospy
from sensor_msgs.msg import JointState
import pandas as pd
import klampt
from klampt import WorldModel
from klampt.model import ik
from klampt import vis
from allegro_control_demo import HandController



In [ ]:
# help(JointState)
state = JointState()

In [ ]:
state.position = [-0.03277025449200767, 0.003013571106191393, 0.006906383669412847, 1.1, -0.010217954488392744, 0.004764243864069904, 0.0010231640262854237, 0.003937912161014311, 0.04439321845272195, 0.008805573227113736, -0.01304334889322642, 0.00889665153955775, 0.2509741202665393, 0.024889682055115662, -0.10084521357662637, -0.07196197865546952]


# import rospy
# from std_msgs.msg import String

test_pub = rospy.Publisher('/allegroHand_0/joint_cmd', JointState, queue_size=10)
rospy.init_node('TestNode')
# r = rospy.Rate(10) # 10hz
# while not rospy.is_shutdown():
#    pub.publish("hello world")
#    r.sleep()

test_pub.publish(state)

# Loading the URDF

In [ ]:
import pandas as pd
import klampt
from klampt import WorldModel
from klampt.model import ik
from klampt import vis
from copy import deepcopy
import numpy as np
from klampt.math import se3,so3

import time
vis.init('PyQt')

active_dofs = [6,7,8,9,11,12,13,14,16,17,18,19,21,22,23,24]

In [ ]:
world = WorldModel()

In [ ]:
world.loadElement('/home/motion/will/ROS_ALLEGRO_HAND/allegro_hand_ros_v4/src/allegro_hand_description/klampt_allegro_hand_right.urdf')

In [ ]:
vis.add('world',world)

vis.show()

In [ ]:
robot = world.robot(0)
zero_config = robot.getConfig()

In [ ]:
for link in range(robot.numLinks()):
    l = robot.link(link)
#     print(l.name)
orig_config = robot.getConfig()


In [ ]:
robot.setConfig(orig_config)

In [ ]:
# a = list(robot.getConfig())
a = deepcopy(orig_config)
a[21] += 1
robot.setConfig(a)

In [ ]:
len(robot.getConfig())

In [ ]:
hand_df = pd.read_pickle('jingchen_hand_run.pkl')
print(hand_df.shape)
# hand_df = hand_df.loc[254:780,:].reset_index(drop = True)
print(hand_df.shape)
pos = hand_df.positions[0]

max_vals = np.zeros(shape = (4,4))
max_vals[:] = -np.inf
min_vals = np.zeros(shape = (4,4))
min_vals[:] = np.inf
for angle in hand_df.angles:
    angle = angle.astype(int)
    max_vals = np.maximum(max_vals,angle)
    min_vals = np.minimum(min_vals,angle)
    
span = max_vals-min_vals
span

In [ ]:
toe_tip = (pos[0][3]-pos[0][0])/1000
toe_tip
pc = klampt.PointCloud()
# for i in range()
from klampt.math import se3,so3
R = so3.from_rotation_vector([0,-np.pi/2+0.25,0])
T = (R,[0,0,-0.2])

# pos = pos.reshape(-1,3)
# for pos in hand_df.positions[100:102]:
#     pc.setPoints(1.4*pos.reshape(-1,3)/1000)
#     pc.transform(R,[-0.04,-0.01,-0.05])
#     points = pc.getPoints()
#     unit_points =
# #     pc.transform(so3.identity(),t)
#     pc
#     vis.add('pc',pc,size = 40,color = [1,0,0,0.5])
#     time.sleep(0.2)


In [ ]:
active_dofs_list = [[11,12,13,14],[6,7,8,9],[16,17,18,19],[21,22,23,24]]
to_state_msg = [6,7,8,9,16,17,18,19,21,22,23,24,11,12,13,14]

In [ ]:
pc = klampt.PointCloud()
for pos,angle in zip(hand_df.positions,hand_df.angles):
    angle = angle.astype(float)
    pc.setPoints(1.4*pos.reshape(-1,3)/1000)
    pc.transform(R,[-0.04,-0.01,-0.05])
    vis.add('pc',pc,size = 40,color = [1,0,0,0.5])
    command = (np.pi/2)*(angle-min_vals)/span
    points = pc.getPoints()
#     robot.setConfig(orig_config)
    prev_config = robot.getConfig()
    toe = robot.link('link_15.0_tip')
    index = robot.link('link_3.0_tip')
    middle = robot.link('link_7.0_tip')
    ring = robot.link('link_11.0_tip')
    objs = []
    obj_toe = ik.objective(toe,local=[0,0,0.017],world=points[3])
#     obj_index = ik.objective(index,local=[0,0,0.017],world=points[7])
#     obj_middle = ik.objective(middle,local=[0,0,0.017],world=points[11])
#     obj_ring = ik.objective(ring,local=[0,0,0.017],world=points[19])
    iks_to_solve = {'toe':[obj_toe,[11,12,13,14]]}#,'index':[obj_index,[6,7,8,9]],
                    #'middle':[obj_middle,[16,17,18,19]],'ring':[obj_ring,[21,22,23,24]]}
#     objs = [obj_toe,obj_index,obj_middle,obj_ring]
    for key in iks_to_solve.keys():
        obj,active_dofs = iks_to_solve[key]
        prev_config = robot.getConfig()
        vis.lock()
        res = ik.solve_global(obj,activeDofs= active_dofs,numRestarts = 10)
        new_config = robot.getConfig()
        robot.setConfig(prev_config)
        if(res):
            robot.setConfig(new_config)
        vis.unlock()
    conf = robot.getConfig()
    conf[7:10] = command[0][1:]
    conf[17:20] = command[1][1:]
    conf[22:25] = command[2][1:]
    conf[21] = 0
    vis.lock()
    robot.setConfig(conf)
    vis.unlock()
    state = JointState()
    state.position = np.array(conf)[[to_state_msg]]
    test_pub.publish(state)
#     a = input()
    
    
    
    
#     print
    #     print(res)
    time.sleep(0.05)
# robot.setConfig(prev_config)


In [ ]:
print(new_config)

In [ ]:
solver.getResidual()

In [ ]:
hand_df

In [ ]:
finger_1.getTransform()

In [ ]:
len(new_config)

In [ ]:
help(vis.add)

# Testing HandController

In [1]:
from allegro_control_demo import HandController, UDP_Server

# import pandas as pd
# import time
# hc = HandController()

# hand_df = pd.read_pickle('jingchen_hand_run.pkl')
# for pos,angle in zip(hand_df.positions,hand_df.angles):
#     hc.animate_and_command(pos,angle)
#     time.sleep(0.033)
server = UDP_Server(record = False,control = False)

***  klampt.vis: using Qt5 as the visualization backend  ***
QtBackend: initializing app as Klamp't visualization
vis: creating GL window
######### QGLWidget setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
UDP server up and listening
######### QGLWidget Initialize GL ###############


In [2]:
server.switch_control()

/home/motion/will/ROS_ALLEGRO_HAND/allegro_hand_ros_v4/allegro_control_demo.py:112: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.state.position = np.array(conf)[[self.to_state_msg]]


#########################################
klampt.vis: Window 0 close
#########################################
